In [ ]:
import math
import numpy as np
import pandas as pd
from numpy import sqrt, exp, pi, power, tanh, vectorize
from scipy.integrate import odeint
import matplotlib.pyplot as plt

In [ ]:
folder = ''

In [ ]:
# time constants: Postnova et al. 2018 - Table 1
tau_v = 50 #s
tau_m = tau_v
tau_H = 59*3600 #s
tau_X = (24*3600) / (2*pi) #s
tau_Y = tau_X
tau_C = 24.2*3600 #s

In [ ]:
# coupling strengths constants: Postnova et al. 2018 - Table 1
nu_vm = -2.1 #mV
nu_mv = -1.8 #mV
nu_Hm = 4.57 #s
nu_Xp = 37*60 #s
nu_Xn = 0.032
nu_YY = (1/3)*nu_Xp #???? OK seems
nu_YX = 0.55*nu_Xp
nu_vH = 1
nu_vC = -0.5 #mV

In [ ]:
# external neuronal drives constants: Postnova et al. 2018 - Table 1
A_v = -10.3 #mV
D_m = 1.3 #mV
A_m = D_m

In [ ]:
# circadian constants: Postnova et al. 2018 - Table 1
gamma = 0.13
delta = 24*3600/0.99729 #s
beta = 0.007/60 #sˆ-1

In [ ]:
# nonphotic drive constant: Postnova et al. 2018 - Table 1
r = 10

In [ ]:
# photic drive constants: Postnova et al. 2018 - Table 1
epsilon = 0.4
I_0 = 100 #lx
I_1 = 9500 #lx
alpha_0 = 0.1/60 #sˆ-1

In [ ]:
# firing rate constants: Postnova et al. 2018 - Table 1
Q_max = 100 #sˆ-1
theta = 10 #mV
sigma = 3 #mV #prime???

In [ ]:
# wake effort constants: Postnova et al. 2018 - Table 1
V_WE = -0.07 #mv
V_th = -2 #mV
F_W = 1 # if forced wake, 0 otherwise #????

In [ ]:
# circadian phase constants: Postnova et al. 2018 - Table 1
MELpeak_t_0 = 0.7*3600 #s
CBTmin_t_0 = 2.7*3600 #s

In [ ]:
# KSS: Karolinska Sleepiness Scale
# Ranges from 1 = "Extremely alert" to 9 = "Extremely sleepy, fighting sleep."
# KSS default parameters: Postnova et al. 2018 - Table 3
Theta_0_KSS = -24.34
Theta_H_KSS = 2.28
Theta_C_KSS = -1.74

In [ ]:
# state function
def state(V_m): # Postnova et al. 2018 - Table 1, Equation 15
    if (V_m > V_th):
        S = 1
    else:
        S = 0
    return S

In [ ]:
# the photoreceptor conversion rate
def alpha(I, S = 1): 
    I = I*S # Postnova et al. 2018 - Table 1, Equation 14
    alpha = ((alpha_0*I)/(I+I_1))*sqrt(I/I_0) # Postnova et al. 2018 - Table 1, Equation 13
    return alpha

In [ ]:
# photic drive to the circadian function
def photic_drive(X, Y, P, I, S = 1):
    D_p = alpha(I,S)*(1-P)*(1-epsilon*X)*(1-epsilon*Y) # Postnova et al. 2018 - Table 1, Equation 12
    return D_p

In [ ]:
# nonphotic drive to the circadian function
def nonphotic_drive(X, S = 1):
    D_n = (S-(2/3))*(1-tanh(r*X)) # Postnova et al. 2018 - Table 1, Equation 11
    return D_n

In [ ]:
# mean population firing rate function
def mean_pop_firing_rate(V):
    Q = Q_max / (1 + exp((theta-V)/sigma)) # Postnova et al. 2018 - Table 1, Equation 7
    return Q

In [ ]:
# total sleep drive
def total_sleep_drive(H,C):
    D_v = nu_vH*H + nu_vC*C + A_v # Postnova et al. 2018 - Table 1, Equation 9
    return D_v

In [ ]:
# Circadian drive, sleep propensity model
def circadian_drive(X,Y):
    X = np.array(X)
    Y = np.array(Y)
    C = 0.1*((1+X)/2)+power(((3.1*X - 2.5*Y + 4.2)/(3.7*(X+2))),2) # Postnova et al. 2016 - Equations 1, 2, and 3
    return C

In [ ]:
# KSS Alertness measure
def alertness_measure_kss(C,H):
    AM_KSS = Theta_0_KSS + Theta_H_KSS*H + Theta_C_KSS*C # Postnova et al. 2018 - Equation 23
    return AM_KSS

In [ ]:
def wake_effort(Q_v):
    W = F_W * max(0, V_WE-nu_mv*Q_v-D_m) # Postnova et al. 2018 - Table 1, Equation 8
    return W

In [ ]:
# test
output = pd.read_csv(folder+"output.csv", sep=";", decimal=",")
input_file = pd.read_csv(folder+'input_irradiance_mel.csv', sep=";", decimal=",")
illuminance_v = input_file[['hours', 'illuminance']]
illuminance_v

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def illuminance(t):
    t = t/3600 # t is received in seconds, we want hours
    t = t % 24
    if ((t < 9.0) or (t > 20.0)): 
        I = 1 # night lux
    else:
        idx = find_nearest(illuminance_v.hours, t)
        I = illuminance_v.illuminance[idx] # factor 5 to get right levels from inputs, find out why inputs so low
    return I

villuminance = vectorize(illuminance)

In [ ]:
### Mean voltages, V_v and V_m of the VLPO and MA
# dV_vdt = (nu_vm*Q_m - V_v + D_v)/tau_v # Postnova et al. 2018 - Table 1, Equation 1
# dV_mdt = (nu_mv*Q_v - V_m + D_m + W)/tau_m # Postnova et al. 2018 - Table 1, Equation 2
### Homeostatic drive
# dHdt = (nu_Hm*Q_m - H)/tau_H # Postnova et al. 2018 - Table 1, Equation 3
### circadian variables
# dXdt = (Y + gamma*(X/3 + power(X,3)*4/3 - power(X,7)*256/105) + nu_Xp*D_p + nu_Xn*D_n)/tau_X # Postnova et al. 2018 - Table 1, Equation 4
# dYdt = (D_p*(nu_YY*Y - nu_YX*X) - power((delta/tau_C),2)*X)/tau_Y # Postnova et al. 2018 - Table 1, Equation 5
### photoreceptor activity function
# dPdt = alpha(I,S)*(1-P)-(beta*P) # Postnova et al. 2018 - Table 1, Equation 6

def model(y, t):
    V_v, V_m, H, X, Y, P = y

    I = illuminance(t)
    S = state(V_m)
    Q_m = mean_pop_firing_rate(V_m)
    Q_v = mean_pop_firing_rate(V_v)
    C = circadian_drive(X,Y)
    D_v = total_sleep_drive(H,C)
    D_n = nonphotic_drive(X, S)
    D_p = photic_drive(X, Y, P, I, S)
    W = wake_effort(Q_v)
    
    gradient_y = [(nu_vm*Q_m - V_v + D_v)/tau_v, # V_v, Postnova et al. 2018 - Table 1, Equation 1
                  (nu_mv*Q_v - V_m + D_m + W)/tau_m, # V_m, Postnova et al. 2018 - Table 1, Equation 2
                  (nu_Hm*Q_m - H)/tau_H, # H, Postnova et al. 2018 - Table 1, Equation 3
                  (Y + gamma*(X/3 + power(X,3)*4/3 - power(X,7)*256/105) + nu_Xp*D_p + nu_Xn*D_n)/tau_X, # X, Postnova et al. 2018 - Table 1, Equation 4
                  (D_p*(nu_YY*Y - nu_YX*X) - power((delta/tau_C),2)*X)/tau_Y, # Y, Postnova et al. 2018 - Table 1, Equation 5
                  alpha(I,S)*(1-P)-(beta*P) # P, Postnova et al. 2018 - Table 1, Equation 6
                 ] 
    return gradient_y

In [ ]:
## https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html
# y0 = [0,0,0,0,0,0]
# initial conditions approximated after reaching some system equilibrium
# y0 = [-2.870465391131989,-0.06999999999999962,14.772935675785682,0.9725400028797555,-0.04658468834644522,2.902957297661591e-21]
y0 = [1.5, -15.0, 13.0, 0.04, -1.28, 0.0]
t = np.linspace(0,48*60*60,17281)
sol = odeint(model,y0,t)
C = circadian_drive(sol[:, 3],sol[:, 4])
AM_KSS = alertness_measure_kss(C,sol[:, 2])

In [ ]:
t_hours = t/3600
plt.plot(t_hours, sol[:, 0], 'b', label='V_v(t)')
plt.plot(t_hours, sol[:, 1], 'g', label='V_m(t)')
plt.plot(t_hours, sol[:, 2], 'r', label='H(t)')
plt.plot(t_hours, sol[:, 3], 'c', label='X(t)')
plt.plot(t_hours, sol[:, 4], 'm', label='Y(t)')
plt.plot(t_hours, sol[:, 5], 'y', label='P(t)')
plt.plot(t_hours, C, 'k', label='C(t)')
plt.plot(t_hours, AM_KSS, 'darkorange', label='A_KSS(t)')
plt.legend(loc='best')
plt.xlabel('t')
#plt.xlim([200000,600000])
plt.grid()
plt.show()

In [ ]:
# get some initial conditions for t = 0
t_temp =round((48*3600/17281)*30)
print(sol[t_temp, 0])
print(sol[t_temp, 1])
print(sol[t_temp, 2])
print(sol[t_temp, 3])
print(sol[t_temp, 4])
print(sol[t_temp, 5])

In [ ]:
plt.plot(output.time, output.KSS, 'b', label='KSS(t)')
plt.plot(t_hours, AM_KSS, 'darkorange', label='AM_KSS(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

# TODO: fix AM_KSS, it should be on [1,9] but goes up to 10

In [ ]:
plt.plot(output.time, output.I_mel, 'b', label='I_mel(t)')
plt.plot(t_hours, villuminance(t), 'darkorange', label='I(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()